In [21]:
from azureml.core import Webservice, Workspace, Dataset, Datastore, Experiment, Run
from azureml.core.model import InferenceConfig, Model
import azureml.dataprep
import math, random, pickle
import pandas as pd
import numpy as np

In [2]:
experiment_name = "titanic_classifier"
model_name = "titanic_classifier_model"
webservice_name = 'titanic-classifier'

In [4]:
from azureml.core.authentication import AzureCliAuthentication

try:
    ws = Workspace.from_config()
except:
    cli_auth = AzureCliAuthentication()

    ws = Workspace(subscription_id="d87d4530-ce4e-4d84-b997-7a78d01e2906",
               resource_group="mlops-RG",
               workspace_name="mlops-AML-WS",
               auth=cli_auth)

Performing interactive authentication. Please follow the instructions on the terminal.
WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...
WARNING - Failed to authenticate '{'additional_properties': {}, 'id': '/tenants/b4c9f32e-da17-4ded-9c95-ce9da38f25d9', 'tenant_id': 'b4c9f32e-da17-4ded-9c95-ce9da38f25d9'}' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS53003: Access has been blocked by Conditional Access policies. The access policy does not allow token issuance.\r\nTrace ID: c1117803-5de5-4871-ba86-055341a33200\r\nCorrelation ID: 08c043a0-b1c1-4f4e-97fe-3005e6054b28\r\nTimestamp: 2020-05-13 20:23:39Z","error_codes":[53003],"timestamp":"2020-05-13 20:23:39Z","trace_id":"c1117803-5de5-4871-ba86-055341a33200","correlation_id

In [5]:
experiment = Experiment(workspace = ws, name = experiment_name)

In [6]:
datastore = Datastore.get_default(workspace=ws)
datastore

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-6f7dfc08-44b5-438d-a6bc-9c804e0bdd76",
  "account_name": "kylemhaleamlsa",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [7]:
titanic_ds = pd.read_csv("./data/titanic3.csv")
#drop unnecessary columns for classifier
titanic_ds.drop(['name','ticket','fare','cabin','embarked','boat','body','home.dest'], axis=1, inplace=True)

#condense Families to unaccompanied boolean
titanic_ds['unaccompanied'] = np.where(titanic_ds.sibsp > 0, 1, np.where(titanic_ds['parch'] > 0, 1, 0))
titanic_ds.drop(['sibsp', 'parch'], axis=1, inplace=True)

#map gender to numeric values
genders = {"male": 0, "female": 1}
titanic_ds['sex'] = titanic_ds['sex'].map(genders)

#round age to nearest year, and fill in missing values with average age
titanic_ds["age"].fillna(titanic_ds["age"].mean(), inplace=True)
titanic_ds['age'] = titanic_ds['age'].astype(int)

#re-bin ages into groups
titanic_ds['age'] = pd.cut(titanic_ds['age'], [-10, 18, 40, 100], labels=[1,2,3]).astype(int)

titanic_ds = titanic_ds.reindex(columns=['pclass','age','sex','unaccompanied','survived'])


In [10]:
engineered_data_path = './data/uploads/titanic-engineered.csv'
titanic_ds.to_csv(engineered_data_path)

datastore.upload(src_dir='./data/uploads', target_path='data')

dataset = Dataset.Tabular.from_delimited_files(datastore.path('data/titanic-engineered.csv'))

titanic_df = dataset.to_pandas_dataframe()

Uploading an estimated of 1 files
Target already exists. Skipping upload for data\titanic-engineered.csv
Uploaded 0 files


In [11]:
from sklearn.model_selection import train_test_split

X = titanic_df.iloc[ : , :-1].values
y = titanic_df.iloc[ : , 4].values

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=39)



In [12]:

from sklearn.tree import DecisionTreeClassifier

run = experiment.start_logging(snapshot_directory=None)

# Log total number of iterations
decision_tree = DecisionTreeClassifier() 
decision_tree.fit(X_train, Y_train)  
Y_pred = decision_tree.predict(X_test) 
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
# Log final results
run.log("Final estimate", acc_decision_tree)
#run.log("Final error",math.pi-pi_estimate)

# Write file containing pi value into run history
#with open("pi_estimate.txt","wb") as f:
    #pickle.dump(str(pi_estimate),f)
filename = 'finalized_model.sav'
pickle.dump(decision_tree , open(filename, 'wb'))
run.upload_file(name = 'outputs/finalized_model.sav', path_or_stream = './finalized_model.sav')

# Complete tracking and get link to details
run.complete()
print("Run completed")

Run completed


In [13]:
from sklearn.metrics import classification_report

print(classification_report(Y_test,Y_pred, target_names=['class 0', 'class 1']))

precision    recall  f1-score   support

     class 0       1.00      1.00      1.00       250
     class 1       1.00      1.00      1.00       182

    accuracy                           1.00       432
   macro avg       1.00      1.00      1.00       432
weighted avg       1.00      1.00      1.00       432



In [15]:
experiment

Name,Workspace,Report Page,Docs Page
titanic_classifier,mlops-AML-WS,Link to Azure Machine Learning studio,Link to Documentation


In [16]:
run = list(Experiment(workspace = ws, name = experiment.name).get_runs())[0]
run

Experiment,Id,Type,Status,Details Page,Docs Page
titanic_classifier,e82e3e4d-f4bd-4f8f-9003-a89e6f2ab78f,,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [17]:
model = run.register_model(model_name = "titanic_classifier", model_path = "outputs/finalized_model.sav")

In [18]:
%%writefile score.py
import pickle, json
from azureml.core.model import Model
import pandas as pd
import numpy as np

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType
from sklearn.externals import joblib

def init():
    global titanic_classifier
    model_path = Model.get_model_path(model_name = "titanic_classifier")
    #model_path = "finalized_model.sav"
    #with open(model_path, "rb") as f:
    titanic_classifier = joblib.load(model_path)

input_sample = pd.DataFrame(data=[{
    "age": 20,
    "pclass": 1,
    "unaccompanied" : 0,
    "sex" : 0
}])

output_sample = np.array([0])

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = titanic_classifier.predict(data)
        return result.tolist()
    except Exception as e:
        result = str(e)
        return error

Overwriting score.py


In [29]:


inference_config = InferenceConfig(entry_script='score.py', runtime='python', conda_file='service-env.yml')


In [34]:
from azureml.core.webservice import AciWebservice

try:
    service = Webservice(ws, "titanic-classifier")
except:
    aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

    service = Model.deploy(workspace=ws, name='titanic-classifier', models=[model], inference_config=inference_config, deployment_config=aci_config, deployment_target=None)
    service.update(description='Binary classifier for Titanic')
    service.wait_for_deployment(show_output = True)

service

service.update(name='titanic-classifier-svc')
print(f'Service State: {service.state}')

TypeError: update() got an unexpected keyword argument 'name'

In [30]:
service.update(models=[model], inference_config = inference_config)

In [31]:
   # Testing our webservice
   
   import json
   from azureml.core import Webservice

   service = Webservice(workspace=ws, name="titanic-classifier")
    
   request = json.dumps({"data" : [{"sex": 0, "pclass" : 3, "age": 70, "unaccompanied": 0}, {"sex": 1, "pclass" : 1, "age": 34, "unaccompanied": 1}]})
   response = service.run(request)
   response #should receive array with 2 predicted values of survival

[1, 1]